In [1]:
library(RPostgreSQL)
library(GenomicRanges)
library(TReNA)
library(FimoClient)
library(RUnit)
library(BSgenome.Hsapiens.UCSC.hg38)
hg38 = BSgenome.Hsapiens.UCSC.hg38
library(SNPlocs.Hsapiens.dbSNP144.GRCh38)
dbSNP <- SNPlocs.Hsapiens.dbSNP144.GRCh38

Loading required package: DBI
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors

Attaching package: ‘S4Vectors’

T

### create and validate external resources, start by testing the fimo microservice

In [2]:
fimo.service <-  FimoClient("whovian", 5558, quiet=TRUE)
test.fimo.service <- function(){
   requestMatch(fimo.service, list(bogus='xxxxx'))
   }
checkEquals(nrow(test.fimo.service()), 0)

[1] TRUE

###  some  fimo-related utility functions  (will evolve in time to proper R package)

In [3]:
load("datalinks/tbl.gwas.level_1.RData")
tmp <- checkEquals(dim(tbl.gwas), c(438609, 9))
source("src/createIgapFimoTrack.R")
tmp <- checkTrue(is.function(doComparativeFimo))

Loading required package: GenomicFeatures
Loading required package: AnnotationDbi
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.



###  now the  gene information (gtf)  database

In [4]:
db.gtf <- dbConnect(PostgreSQL(), user= "trena", password="trena", dbname="gtf", host="whovian")
tmp <- checkEquals(dbListTables(db.gtf), "hg38human")
query <- "select count(*) from hg38human where moleculetype='gene' and gene_biotype='protein_coding'"
tmp <- checkTrue(dbGetQuery(db.gtf, query)$count > 19500)


In [5]:
load("datalinks/tbl.gwas.level_1.RData")
tmp <- checkTrue(nrow(tbl.gwas) > 400000)

### initialize the FootprintFinder

In [6]:
genome.db.uri    <- "postgres://whovian/hg38"                  # has gtf and motifsgenes tables
footprint.db.uri <- "postgres://whovian/brain_hint"            # has hits and regions tables
fpf <- FootprintFinder(genome.db.uri, footprint.db.uri, quiet=FALSE)

[1] postgres://whovian/hg38/gtf: 2568100 rows
[1] postgres://whovian/hg38/motifsgenes: 9289 rows
[1] postgres://whovian/brain_hint/regions: 22772585 rows


In [7]:
mef2c.proximal.fp <- getFootprintsForGene(fpf, "MEF2C", size.upstream=1000, size.downstream=1000)
tmp <- checkTrue(nrow(mef2c.proximal.fp) > 10)

$chr
[1] "chr5"

$start
[1] 88903257

$end
[1] 88905257



### goi:  genes of interest

In [8]:
goi <- c("SNRNP70", "SNRPA", "SNRPC", "SMN1", "SNRPB", "PLCD1",                     # from emory, sinai, ufl
         "PTRHD1", "SFRP1", "PPP1R7", "DNM3", "RTN4", "EPB41L3", "TUBB3",
         "PLEC", "ANXA5", "MSN", "CD44", "LMNA", "DOCK2", "GABBR2", "GABRB2",
         "GIGYF1", "ITGB2", "JPH3", "LAPTM5", "NCKAP1L", "OPCML", "RBM3",
         "SCAMP1", "SCN2A", "SELT", "SNAP25", "SNAP91", "STXBP1", "SUB1",
         "SYT1", "TARBP1", "YWHAG", "TGFBR1", "BMPR1A", "BMPR1B", "VGF", "CRH",
         "TREM2", "TYROBP", "S100A8", "S100A9", "P2RY2", "P2RX7", "P2RY12",
         "P2RY13", "OSMR", "TLR4", "CR1", "CSF1R", "CX3CR1", "SPI1",
         "TNFRSF10A", "TNFRSF10B", "MEF2C",
         "HS3ST1", "SQSTM1", "TREML2", "NDUFAF6", "ECHDC3", "AP2A2", "ADAMTS20",   # from igap paper
         "SPPL2A", "TRIP4", "SCIMP", "ACE")

###  tbl.genes  has all the information needed on  the goi

In [9]:
query <- "select * from hg38human where moleculetype='gene' and gene_biotype='protein_coding'"
tbl <- dbGetQuery(db.gtf, query) [, c("chr", "gene_name", "start", "endpos", "strand")]
tbl.genes <- subset(tbl, gene_name %in% goi)
tbl.genes$TSS <- tbl.genes$start
minus.strand.genes <- which(tbl.genes$strand=='-')
tbl.genes$TSS[minus.strand.genes] <- tbl.genes$endpos[minus.strand.genes]

###  find snps within 'shoulder' distance of each gene's TSS

In [10]:
shoulder <- 8000
gr.genes <- with(tbl.genes, GRanges(seqnames=chr, IRanges(start=TSS-shoulder, end=TSS+shoulder)))
gr.snps   <- with(tbl.gwas, GRanges(seqnames=CHR, IRanges(start=BP, end=BP)))
suppressWarnings(
    tbl.overlaps <- as.data.frame(findOverlaps(gr.genes, gr.snps, type="any"))
    )
tbl.combined <- cbind(tbl.genes[tbl.overlaps$queryHits,], tbl.gwas[tbl.overlaps$subjectHits,])
print(table(tbl.combined$gene_name))


      ACE  ADAMTS20     AP2A2    BMPR1B      CD44       CR1       CRH     CSF1R 
       16         2        31         6         1        18         1         8 
    DOCK2    ECHDC3   EPB41L3    GABRB2    HS3ST1    LAPTM5     MEF2C   NCKAP1L 
        3         3         1         1         2        14        10         7 
  NDUFAF6     P2RX7    P2RY12    P2RY13     PLCD1    PPP1R7    PTRHD1      RTN4 
       17         1        11        12         1        11        19         1 
    SCIMP     SCN2A    SNAP25    SNAP91      SPI1    SPPL2A    SQSTM1      TLR4 
       19         5         2         1        15        16        11         6 
TNFRSF10A     TREM2    TREML2     TRIP4     TUBB3       VGF     YWHAG 
        8         4        11         3         3        11         4 


### create a table of footprints around the tss of each gene in tbl.combined  (slow!  ~ 2 minutes)

In [11]:
tbl.fp <- data.frame()
for(gene in unique(tbl.combined$gene_name)){
    #printf("--- gene: %s", gene)
    tbl.fpForGene <- getFootprintsForGene(fpf, gene, size.upstream=1000, size.downstream=1000)
    #printf("%d fps for %s", nrow(tbl.fpForGene), gene)
    tbl.fp <- rbind(tbl.fp, tbl.fpForGene)
    }
dim(tbl.fp)

$chr
[1] "chr1"

$start
[1] 30756820

$end
[1] 30758820

$chr
[1] "chr1"

$start
[1] 207495147

$end
[1] 207497147

$chr
[1] "chr2"

$start
[1] 24792382

$end
[1] 24794382

$chr
[1] "chr2"

$start
[1] 55111621

$end
[1] 55113621

$chr
[1] "chr2"

$start
[1] 165238402

$end
[1] 165240402

$chr
[1] "chr2"

$start
[1] 241148576

$end
[1] 241150576

$chr
[1] "chr3"

$start
[1] 38028762

$end
[1] 38030762

$chr
[1] "chr3"

$start
[1] 151328548

$end
[1] 151330548

$chr
[1] "chr3"

$start
[1] 151383812

$end
[1] 151385812

$chr
[1] "chr4"

$start
[1] 11428765

$end
[1] 11430765

$chr
[1] "chr4"

$start
[1] 94756968

$end
[1] 94758968

$chr
[1] "chr5"

$start
[1] 88903257

$end
[1] 88905257

$chr
[1] "chr5"

$start
[1] 150112372

$end
[1] 150114372

$chr
[1] "chr5"

$start
[1] 161548044

$end
[1] 161550044

$chr
[1] "chr5"

$start
[1] 169636247

$end
[1] 169638247

$chr
[1] "chr5"

$start
[1] 179805398

$end
[1] 179807398

$chr
[1] "chr6"

$start
[1] 41162186

$end
[1] 41164186

$chr
[1] "chr

[1] 11509    17

###  within these gene promoter regions, now find overlap of footprints and  snps

In [12]:
gr.snpsInPromoters <- with(tbl.combined, GRanges(seqnames=chr, IRanges(start=BP, end=BP)))
gr.fpInPromoters   <- with(tbl.fp, GRanges(seqnames=chrom, IRanges(start=start-10, end=endpos+10)))
tbl.ov2 <- suppressWarnings(as.data.frame(findOverlaps(gr.snpsInPromoters, gr.fpInPromoters, type="any")))

tbl.snpsInPromotersInFootprints <- cbind(tbl.combined[tbl.ov2$queryHits,], tbl.fp[tbl.ov2$subjectHits,])
print(unique(tbl.snpsInPromotersInFootprints$gene_name))

 [1] "LAPTM5"    "SCN2A"     "PPP1R7"    "P2RY13"    "BMPR1B"    "DOCK2"    
 [7] "SQSTM1"    "TNFRSF10A" "NDUFAF6"   "TLR4"      "AP2A2"     "SPPL2A"   
[13] "SCIMP"     "ACE"       "SNAP25"   


### create tbl.prospects,  a condensed table of gene + snp info

In [13]:
gene.prospects <- unique(tbl.snpsInPromotersInFootprints$gene_name)
tbl.prospects <- data.frame()
for(gene in gene.prospects){
   tbl.new <- unique(subset(tbl.snpsInPromotersInFootprints, gene_name==gene)[, c("gene_name", "SNP", "chr", "BP", "strand")])
   tbl.prospects <- rbind(tbl.prospects, tbl.new)
   }

### iterate over prospects, identify the promoter/footprint snps which change FIMO predictions  (~  90 seconds )

In [14]:
# make sure the fimo service is up and running.  
result <- requestMatch(fimo.service, list(bogus='xxxxx'))
tmp <- checkEquals(result, data.frame())

In [15]:
tbl.prospects

,gene_name,SNP,chr,BP,strand
430.8,LAPTM5,rs2273979,chr1,30757821,-
2827,SCN2A,rs6718960,chr2,165239218,+
3279.6,PPP1R7,rs7596642,chr2,241150035,+
3279.7,PPP1R7,rs7584799,chr2,241150158,+
4130.5,P2RY13,rs3732757,chr3,151328816,-
4801.2,BMPR1B,rs4348103,chr4,94757292,+
5856,DOCK2,rs174658,chr5,169637710,+
5958.8,SQSTM1,rs73351610,chr5,179805603,+
8872.6,TNFRSF10A,rs79037040,chr8,23225458,-
9168.4,NDUFAF6,rs77733982,chr8,94894935,+


In [16]:
for(r in 1:nrow(tbl.prospects)){
  rsid <- tbl.prospects$SNP[r]
  if(rsid == "rs79037040") rsid <- "rs13278062"
  chrom <- tbl.prospects$chr[r]
  loc <- tbl.prospects$BP[r]
  gene <- tbl.prospects$gene_name[r]
  ambiguity.code <- snpsById(dbSNP, rsid)$alleles_as_ambig
  elements.string <- IUPAC_CODE_MAP[[ambiguity.code]]
  elements <- strsplit(elements.string,'')[[1]]
  wt <- as.character(getSeq(hg38, chrom, loc, loc))
  mut <- setdiff(elements, wt)
  status <- doComparativeFimo(chrom, loc, wt, mut, 10, quiet=TRUE)
  printf("---- %s: %s", gene, status)
  }

[1] ---- LAPTM5: noMotif
[1] ---- SCN2A: gain
[1] ---- PPP1R7: noMotif
[1] ---- PPP1R7: noMotif
[1] ---- P2RY13: noMotif
[1] ---- BMPR1B: lossAndGain


Warning message:
In if (mut.sequence == wt.sequence) {: the condition has length > 1 and only the first element will be used

[1] ---- DOCK2: gain
[1] ---- SQSTM1: lossAndGain
[1] ---- TNFRSF10A: noMotif
[1] ---- NDUFAF6: noMotif
[1] ---- NDUFAF6: lossAndGain
[1] ---- NDUFAF6: gain
[1] ---- NDUFAF6: gain
[1] ---- TLR4: noMotif
[1] ---- AP2A2: gain
[1] ---- AP2A2: gain
[1] ---- AP2A2: gain
[1] ---- SPPL2A: noMotif
[1] ---- SCIMP: noMotif
[1] ---- ACE: noMotif
[1] ---- ACE: noMotif
[1] ---- SNAP25: gain


cory: AP2A2 is interesting because it could easily be tied into APOE (it's also a lipid binding protein).
NDUFAF6 is of potential interest because it appears to be pretty central to glucose metabolism, which has been shown to change in people who later develop AD. 
